In [ ]:
import asyncio
import numpy as np
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK

In [ ]:
async def run_bump_test(csc, sleep_time=5.0):
    """Run the bump test.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    force : float
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    # Print the information of inclinometer
    zenithAngle = csc.tel_zenithAngle.get()
    print(f"Measured zenith angle: {zenithAngle.measured:0.2f} degree.")
    print(f"Raw inclinometer angle: {zenithAngle.inclinometerRaw:0.2f} degree.")
    print(f"Processed inclinometer angle: {zenithAngle.inclinometerProcessed:0.2f} degree.")

    num_axial_actuator = np.array([9, 32, 70])
    force = [-150, -100, -50, +50, +100, +150]

    
    for idx in num_axial_actuator:
        print(f"Do the bump test of actuator: {idx}.")
        for f in force:
            await bump_axial_actuator(csc, idx, f, sleep_time=sleep_time)


In [ ]:
async def bump_axial_actuator(csc, idx_actuator, force, sleep_time=5):
    """Bump the axial actuator.
    
    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    idx_actuator : int
        Index of actuator.
    force : float
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    num_axial_actuator = NUM_ACTUATOR - NUM_TANGENT_LINK
    forces = [0] * num_axial_actuator
    forces[idx_actuator] = force

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(sleep_time)

    
    print(f"Apply the force: {force} N.")
    await csc.cmd_applyForces.set_start(axial=forces)
    await asyncio.sleep(sleep_time)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(sleep_time)


In [ ]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [ ]:
state_m2 = m2.evt_summaryState.get()
if state_m2.summaryState != salobj.State.ENABLED:
    await salobj.set_summary_state(m2, salobj.State.ENABLED, timeout=60)

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

In [ ]:
# bump actuators with linear sequence of forces

await run_bump_test(m2, sleep_time=5)

In [ ]:
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")